Complex query should have from 3 or more tables joined, a custom scalar function, and use built-in SQL functions and group by summarization. It should include combinations of subqueries or CTE or virtual tables.

  

7 Complex Queries

 **Complex Prompt 1 :** 

The HR department wants to review the compensation structure for the employees over the past year. They aim to understand how monthly salaries have been determined and to gain insights into the average monthly salaries across different departments and shifts. Additionally, they wish to view the average monthly vacation and sick leave usage per department and shift.

**Database :** AdventureWorks2017

**Tables Involved :** <span style="color: rgb(33, 33, 33); font-family: Consolas, &quot;Courier New&quot;, monospace; font-size: 12px; white-space: pre;">HumanResources.Shift,</span>

HumanResources.Employee, HumanResources.EmployeeDepartmentHistory,

HumanResources.Department, HumanResources.EmployeePayHistory

**CTE :** MonthlyRateChangeCTE

**FUNCTION :** MonthlySalary : Computes Monthly salary based on provided rate and pay frequency

In [5]:
-- Check if the function `dbo.fn_MonthlySalary` exists in the database.
-- If it does, then we will proceed to drop it.
IF EXISTS (SELECT 1 FROM sys.objects WHERE object_id = OBJECT_ID(N'dbo.fn_MonthlySalary') AND type = 'FN')
DROP FUNCTION dbo.fn_MonthlySalary;
GO

-- Define and create a new function `dbo.fn_MonthlySalary`
-- This function computes the monthly salary based on provided rate and pay frequency.
    -- Using a CASE statement, determine the monthly salary.
    -- If PayFrequency is 1, it's assumed to be Monthly, hence salary is same as rate.
    -- If PayFrequency is 2, it's assumed to be Bi-Weekly, hence salary is twice the rate.
    -- For any other value, the monthly salary is assumed to be the rate itself.
CREATE FUNCTION fn_MonthlySalary (@Rate decimal(10,2), @PayFrequency int)
RETURNS decimal(10,2) 
AS 
BEGIN 
    DECLARE @MonthlySalary decimal(10,2)
    
    SET @MonthlySalary = CASE 
                             WHEN @PayFrequency = 1 THEN @Rate
                             WHEN @PayFrequency = 2 THEN @Rate * 2
                             ELSE @Rate
                         END
    RETURN @MonthlySalary
END;
GO

-- Switch to the `AdventureWorks2017` database for subsequent operations.
USE AdventureWorks2017;
GO

-- Using a CTE named `MonthlyRateChangeCTE`, compute the average rate change for employees over the past year.
WITH MonthlyRateChangeCTE AS
(
    SELECT
        eph.BusinessEntityID,
        AVG(eph.Rate) AS AverageMonthlyRateChange
    FROM HumanResources.EmployeePayHistory eph
    WHERE eph.RateChangeDate > DATEADD(YEAR, -1, GETDATE())
    GROUP BY eph.BusinessEntityID
)

-- The main query

-- - Department and Shift Names
-- - Average Monthly Salary
-- - Average Monthly Vacation and Sick Hours
SELECT 
    d.Name AS DepartmentName,
    s.Name AS ShiftName,
    AVG(dbo.fn_MonthlySalary(eph.Rate, eph.PayFrequency)) AS AverageMonthlySalary,
    AVG(e.VacationHours/12.0) AS AverageMonthlyVacationHours,
    AVG(e.SickLeaveHours/12.0) AS AverageMonthlySickHours
FROM HumanResources.Employee e
JOIN HumanResources.EmployeeDepartmentHistory edh ON e.BusinessEntityID = edh.BusinessEntityID
JOIN HumanResources.Department d ON edh.DepartmentID = d.DepartmentID
JOIN HumanResources.EmployeePayHistory eph ON e.BusinessEntityID = eph.BusinessEntityID
JOIN HumanResources.Shift s ON edh.ShiftID = s.ShiftID
LEFT JOIN MonthlyRateChangeCTE mrc ON e.BusinessEntityID = mrc.BusinessEntityID
GROUP BY d.Name, s.Name;


Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

(29 rows affected)

Total execution time: 00:00:00.177

DepartmentName,ShiftName,AverageMonthlySalary,AverageMonthlyVacationHours,AverageMonthlySickHours
Document Control,Day,25.526666,6.499999,4.888888
Engineering,Day,67.344444,1.527777,3.416666
Executive,Day,136.610000,2.062500,2.687499
Facilities and Maintenance,Day,36.146666,7.361110,5.333333
Finance,Day,59.549230,3.814102,3.557691
Human Resources,Day,36.050000,4.291666,3.791666
Information Services,Day,68.691111,5.787036,4.537036
Marketing,Day,37.870000,3.708333,3.499999
Production,Day,16.056632,3.787414,3.539115
Production Control,Day,34.066666,3.722222,3.499999


 **Complex Prompt 2 :** 

You are an analyst at PrestigeCars, a renowned car dealership with a significant sales volume. PrestigeCars has detailed records of its sales across several years. As the business grew, the sales records were maintained in separate tables for each year in the DataTransfer schema.

The executive team at PrestigeCars is interested in a consolidated view of sales across these years. They wish to understand:

The maximum sale price for each car model sold to a customer from a specific country.

The average profit earned from each model, given the various costs associated with getting the car ready for sale.

For strategic reasons, the team is particularly interested in models where the average profit is greater than $5000

**Database :** PrestigeCars

**Tables Involved** :  
DataTransfer.Sales2015, DataTransfer.Sales2016,

DataTransfer.Sales2017, DataTransfer.Sales2018,

ConsolidatedSales , Data.Customer 

**CTE :** <span style="color: rgb(33, 33, 33); font-family: Consolas, &quot;Courier New&quot;, monospace; font-size: 12px; white-space: pre;">ConsolidatedSales</span>

**FUNCTION :** <span style="color: rgb(33, 33, 33); font-family: Consolas, &quot;Courier New&quot;, monospace; font-size: 12px; white-space: pre;">fnCalculateProfit</span>: Computes Monthly salary based on provided rate and pay frequency

In [4]:
-- Create a function `dbo.fnCalculateProfit` to compute profit.
-- It takes the sale price and total cost as input parameters.

-- Check if the function exists and if so, drop it.
IF EXISTS (SELECT 1 FROM sys.objects WHERE object_id = OBJECT_ID(N'dbo.fnCalculateProfit') AND type = 'FN')
DROP FUNCTION dbo.fnCalculateProfit;
GO

-- Define the function to calculate profit as the difference between sale price and total cost.
CREATE FUNCTION dbo.fnCalculateProfit(@SalePrice DECIMAL, @TotalCost DECIMAL)
RETURNS DECIMAL
AS
BEGIN
    RETURN (@SalePrice - @TotalCost)
END;
GO

-- Set the context to the `PrestigeCars` database.
USE PrestigeCars;

-- Using a CTE named `ConsolidatedSales`, consolidate sales data from different years.
-- For each year, compute the total cost associated with each car sale by summing up various cost components.
WITH ConsolidatedSales AS (
    SELECT 
        MakeName, ModelName, CustomerName, CountryName, 
        Cost + RepairsCost + PartsCost + TransportInCost AS TotalCost, 
        SalePrice, SaleDate 
    FROM DataTransfer.Sales2015
    UNION ALL
    SELECT 
        MakeName, ModelName, CustomerName, CountryName, 
        Cost + RepairsCost + PartsCost + TransportInCost AS TotalCost, 
        SalePrice, SaleDate 
    FROM DataTransfer.Sales2016
    UNION ALL
    SELECT 
        MakeName, ModelName, CustomerName, CountryName, 
        Cost + RepairsCost + PartsCost + TransportInCost AS TotalCost, 
        SalePrice, SaleDate 
    FROM DataTransfer.Sales2017
    UNION ALL
    SELECT 
        MakeName, ModelName, CustomerName, CountryName, 
        Cost + RepairsCost + PartsCost + TransportInCost AS TotalCost, 
        SalePrice, SaleDate 
    FROM DataTransfer.Sales2018
)

-- The main query retrieves the desired details using the above CTE.
-- Join the consolidated sales with the `Data.Customer` table.
-- Group the results by country, customer, car make, and model.
-- Filter out results where average profit is less than or equal to $5000.
-- Order the results by average profit in descending order.
SELECT 
    CS.CountryName,
    CS.CustomerName,
    CS.MakeName,
    CS.ModelName,
    MAX(CS.SalePrice) AS MaxSalePrice,
    AVG(dbo.fnCalculateProfit(CS.SalePrice, CS.TotalCost)) AS AvgProfit
FROM 
    ConsolidatedSales CS
JOIN
    Data.Customer C
    ON CS.CustomerName = C.CustomerName
GROUP BY 
    CS.CountryName, 
    CS.CustomerName, 
    CS.MakeName, 
    CS.ModelName
HAVING 
    AVG(dbo.fnCalculateProfit(CS.SalePrice, CS.TotalCost)) > 5000 -- Filter: Only cars with average profit more than 5000
ORDER BY 
    AvgProfit DESC;


Commands completed successfully.

Commands completed successfully.

(171 rows affected)

Total execution time: 00:00:00.126

CountryName,CustomerName,MakeName,ModelName,MaxSalePrice,AvgProfit
France,Vive La Vitesse,Ferrari,Enzo,365000.00,66350.000000
France,Laurent Saint Yves,Ferrari,Enzo,395000.00,65600.000000
United Kingdom,Birmingham Executive Prestige Vehicles,Bugatti,57C,335000.00,60850.000000
France,Bling Bling S.A.,Bugatti,57C,345000.00,60800.000000
United Kingdom,Andrea Tarbuck,Bugatti,57C,355000.00,57600.000000
France,La Bagnole de Luxe,Bugatti,57C,355000.00,52300.000000
Germany,Glitz,Lamborghini,Jarama,305000.00,51950.000000
United Kingdom,Honest Pete Motors,Bugatti,57C,295000.00,50800.000000
United Kingdom,Kieran O'Harris,Lamborghini,Diabolo,255000.00,46300.000000
Spain,Alicia Almodovar,Ferrari,Testarossa,250000.00,45350.000000


 **Complex Prompt 3 :** 

WideWorldImporters, a global trading company, has always taken pride in the strength and reliability of its supplier network. Recognizing that long-standing relationships have been a cornerstone of their success, the company's procurement division decides to conduct an analysis. Their objective is to identify and evaluate the top suppliers with whom they have transacted for over a decade, gauging them based on their order volumes, associated product categories, average transaction amounts, and recent transaction frequency.

**Database :** PrestigeCars

**Tables Involved** :

Purchasing.SupplierTransactions, Purchasing.PurchaseOrders,  
Purchasing.PurchaseOrderLines, Warehouse.StockItems, Warehouse.StockItemStockGroups, Warehouse.StockGroups, Purchasing.Suppliers, Purchasing.PurchaseOrders, Purchasing.PurchaseOrderLines

**CTE :** <span style="color: rgb(33, 33, 33); font-family: Consolas, &quot;Courier New&quot;, monospace; font-size: 12px; white-space: pre;">ConsolidatedSales</span>

**FUNCTION :** <span style="color: rgb(33, 33, 33); font-family: Consolas, &quot;Courier New&quot;, monospace; font-size: 12px; white-space: pre;">dbo.fnYearDiff </span> : Compute the difference between two dates in years

In [35]:
-- Set the current database context to WideWorldImporters.
USE WideWorldImporters;
GO

-- Check if the custom function named 'dbo.fnYearDiff' already exists in the database.
-- If it does, we drop it to ensure our new definition is the one being used.
IF EXISTS (SELECT 1 FROM sys.objects WHERE object_id = OBJECT_ID(N'dbo.fnYearDiff') AND type = 'FN')
DROP FUNCTION dbo.fnYearDiff;
GO

-- Create a user-defined function 'dbo.fnYearDiff' to compute the difference between two dates in years.
CREATE FUNCTION dbo.fnYearDiff(@StartDate DATE, @EndDate DATE)
RETURNS INT 
AS 
BEGIN 
    -- Calculate the difference in years, considering the month and day to be precise.
    RETURN (YEAR(@EndDate) - YEAR(@StartDate) + 
           CASE 
               WHEN MONTH(@EndDate) > MONTH(@StartDate) OR (MONTH(@EndDate) = MONTH(@StartDate) AND DAY(@EndDate) >= DAY(@StartDate)) THEN 1
               ELSE 0
           END)
END;
GO

-- Use a Common Table Expression (CTE) named 'TopSuppliers' to determine the top 10 suppliers 
-- based on the total volume of items ordered.
WITH TopSuppliers AS
(
    SELECT TOP (10)
        s.SupplierId,
        s.SupplierName,
        -- Aggregate the total ordered items for each supplier.
        SUM(pol.OrderedOuters) AS TotalOrderedItems
    FROM Purchasing.Suppliers s
    -- Join with PurchaseOrders and PurchaseOrderLines tables to access order details.
    JOIN Purchasing.PurchaseOrders po ON s.SupplierID = po.SupplierID
    JOIN Purchasing.PurchaseOrderLines pol ON po.PurchaseOrderID = pol.PurchaseOrderID
    GROUP BY s.SupplierId, s.SupplierName
    ORDER BY TotalOrderedItems DESC
)

-- The main query extracts detailed metrics for each of these top suppliers.
SELECT 
    ts.SupplierName,
    sg.StockGroupName,
    -- Compute the average transaction amount for each supplier-stock group combination.
    AVG(st.TransactionAmount) AS AverageTransactionAmount,
    -- Count distinct transactions to get the frequency.
    COUNT(DISTINCT st.SupplierTransactionID) AS RecentTransactionCount
FROM 
    TopSuppliers ts
-- Join with multiple tables to access transaction details, stock items, stock groups, etc.
JOIN Purchasing.SupplierTransactions st ON ts.SupplierId = st.SupplierId
JOIN Purchasing.PurchaseOrders po ON st.PurchaseOrderID = po.PurchaseOrderID
JOIN Purchasing.PurchaseOrderLines pol ON po.PurchaseOrderID = pol.PurchaseOrderID
JOIN Warehouse.StockItems si ON pol.StockItemID = si.StockItemID
JOIN Warehouse.StockItemStockGroups sisg ON si.StockItemID = sisg.StockItemID
JOIN Warehouse.StockGroups sg ON sisg.StockGroupID = sg.StockGroupID
-- Use the created function to filter transactions that happened more than 10 years ago.
WHERE 
    dbo.fnYearDiff(st.TransactionDate, GETDATE()) > 10
GROUP BY 
    ts.SupplierName, 
    sg.StockGroupName
-- Order results by average transaction amount in descending order.
ORDER BY 
    AverageTransactionAmount DESC;


Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

(15 rows affected)

Total execution time: 00:00:00.159

SupplierName,StockGroupName,AverageTransactionAmount,RecentTransactionCount
"Fabrikam, Inc.",T-Shirts,170438.985124,248
"Fabrikam, Inc.",Computing Novelties,170438.985124,248
"Fabrikam, Inc.",Clothing,164988.106014,248
"Litware, Inc.",Packaging Materials,25214.132187,212
Northwind Electric Cars,Toys,21569.790555,10
Northwind Electric Cars,Novelty Items,21569.790555,10
"Fabrikam, Inc.",Novelty Items,16721.216250,4
"Fabrikam, Inc.",Furry Footwear,11966.326666,7
The Phone Company,Computing Novelties,10876.166000,5
The Phone Company,Novelty Items,10876.166000,5


**Complex Prompt 4 :**   
Given the AdventureWorksDW2017 database, we want to analyze the performance of the resellers in various regions for the top 5 products in 2013. The aim is to determine which resellers have the highest sales amounts for these top products and to calculate a "Loyalty Score" for each reseller based on their years of purchase and average sales amount. Provide a list of resellers, their respective regions, the top product they've sold, their total sales amount for that product, and the calculated Loyalty Score.

**Database :** AdventureWorksDW2017 

**Tables Involved** :  
dd.FactResellerSales, dd.DimProduct, dd.DimDate, 

dp.ProductKey, YearlyTopProducts, dbo.FactResellerSales, 

dbo.DimReseller, dbo.DimGeography

**CTE :** YearlyTopProducts 

**FUNCTION :** fnCalculateLoyaltyScore

In [56]:
-- Setting the database context to AdventureWorksDW2017.
USE AdventureWorksDW2017;
GO

-- If you haven't created the function yet or dropped it earlier, you can create it again with this block.
-- Create a user-defined function to compute the Loyalty Score.
CREATE FUNCTION dbo.fnCalculateLoyaltyScore (@YearsOfPurchase INT, @AvgAnnualSalesAmount DECIMAL)
RETURNS DECIMAL
AS
BEGIN
    RETURN @YearsOfPurchase * @AvgAnnualSalesAmount
END;
GO

-- Use a Common Table Expression (CTE) named 'YearlyTopProducts' 
-- to determine the top 5 products based on sales for the year 2013.
WITH YearlyTopProducts AS
(
    SELECT TOP (5)
        dp.ProductKey,
        dp.ModelName,
        SUM(frs.SalesAmount) AS AnnualSalesAmount
    FROM dbo.FactResellerSales frs
    JOIN dbo.DimProduct dp ON frs.ProductKey = dp.ProductKey
    JOIN dbo.DimDate dd ON frs.OrderDateKey = dd.DateKey
    WHERE dd.CalendarYear = 2013
    GROUP BY dp.ProductKey, dp.ModelName
    ORDER BY AnnualSalesAmount DESC
)

-- Main query to identify resellers' contribution to the sales of these top products in various regions.
SELECT 
    dg.StateProvinceName,
    dg.CountryRegionCode,
    tp.ModelName AS TopProduct,
    dr.ResellerName,
    SUM(frs.SalesAmount) AS ResellerSalesAmount,
    -- Calculate Loyalty Score using the custom function.
    dbo.fnCalculateLoyaltyScore(DATEDIFF(YEAR, MIN(frs.OrderDate), GETDATE()), AVG(frs.SalesAmount)) AS LoyaltyScore
FROM 
    YearlyTopProducts tp
-- Join with other relevant tables to gather geographic and reseller details.
JOIN dbo.FactResellerSales frs ON tp.ProductKey = frs.ProductKey
JOIN dbo.DimReseller dr ON frs.ResellerKey = dr.ResellerKey
JOIN dbo.DimGeography dg ON dr.GeographyKey = dg.GeographyKey
GROUP BY 
    dg.StateProvinceName, 
    dg.CountryRegionCode, 
    tp.ModelName,
    dr.ResellerName
ORDER BY 
    ResellerSalesAmount DESC,
    dg.StateProvinceName,
    dg.CountryRegionCode;


Commands completed successfully.

: Msg 2714, Level 16, State 3, Procedure fnCalculateLoyaltyScore, Line 4
There is already an object named 'fnCalculateLoyaltyScore' in the database.

(313 rows affected)

Total execution time: 00:00:00.098

StateProvinceName,CountryRegionCode,TopProduct,ResellerName,ResellerSalesAmount,LoyaltyScore
California,US,Touring-1000,Westside Plaza,103498.4389,142307
Loiret,FR,Mountain-200,Registered Cycle Store,89194.3274,111490
Arizona,US,Touring-1000,Rally Master Company Inc,84801.1315,116600
Garonne (Haute),FR,Touring-1000,Perfect Toys,83426.2384,119180
England,GB,Mountain-200,Metropolitan Bicycle Supply,80972.9847,101220
Wyoming,US,Mountain-200,Great Bikes,79410.326,124784
Colorado,US,Mountain-200,Field Trip Store,77776.9816,97220
Alberta,CA,Mountain-200,Top Sports Supply,75734.67,104137
Washington,US,Mountain-200,Safe Cycles Shop,73487.4158,91860
Utah,US,Mountain-200,Brakes and Gears,73066.7441,104380


**Complex Prompt 5 :**  
In the NorthWinds2022TSQLV7 database, we have information regarding employees, products, suppliers, customers, orders, and more. We want to analyze the sales performance of each employee for the year 2022. Specifically, find the total number of products sold, total sales amount (considering discounts), and the sales commission for each employee based on their total sales. The sales commission is calculated as 5% of the total sales amount. Additionally, list the region where the employee is located. Order the result based on the total sales amount in descending order.

**Database :**  NorthWinds2022TSQLV7

**Tables Involved** :

Sales.\[Order\], Sales.OrderDetail, HumanResources.Employee, o.OrderID

**CTE :** SalesDetails

**FUNCTION :** <span style="font-family: Consolas, &quot;Courier New&quot;, monospace; font-size: 12px; white-space: pre; color: rgb(121, 94, 38);">dbo</span><span style="color: rgb(33, 33, 33); font-family: Consolas, &quot;Courier New&quot;, monospace; font-size: 12px; white-space: pre;">.fnCalculateSalesCommission : </span> <span style="color: rgb(0, 128, 0); font-family: Consolas, &quot;Courier New&quot;, monospace; font-size: 12px; white-space: pre;">compute the sales commission based on total sales amount.</span>

In [67]:
-- Set the active database to NorthWinds2022TSQLV7
USE NorthWinds2022TSQLV7;
GO

-- Check if the function `dbo.fnCalculateSalesCommission` exists in the database.
-- If it exists, drop it.
IF EXISTS (SELECT 1 FROM sys.objects WHERE object_id = OBJECT_ID(N'dbo.fnCalculateSalesCommission') AND type = 'FN')
DROP FUNCTION dbo.fnCalculateSalesCommission;
GO

-- Create a user-defined function to compute the sales commission based on total sales amount.
CREATE FUNCTION dbo.fnCalculateSalesCommission (@TotalSales DECIMAL(18,2))
RETURNS DECIMAL(18,2)
AS
BEGIN
    RETURN @TotalSales * 0.05
END;
GO

-- Using a Common Table Expression (CTE) to compute sales details for each order and product.
WITH SalesDetails AS
(
    SELECT 
        o.EmployeeID,
        od.ProductID,
        od.UnitPrice * od.Quantity * (1 - od.DiscountPercentage / 100) AS SalesAmount
    FROM Sales.[Order] o
    JOIN Sales.OrderDetail od ON o.OrderID = od.OrderID
    WHERE YEAR(o.OrderDate) = 2014
)

-- Main query to fetch the sales performance of each employee.
SELECT 
    e.EmployeeFirstName + ' ' + e.EmployeeLastName AS EmployeeName,
    COUNT(DISTINCT sd.ProductID) AS NumberOfProductsSold,
    SUM(sd.SalesAmount) AS TotalSalesAmount,
    -- Use the user-defined function to calculate sales commission.
    dbo.fnCalculateSalesCommission(SUM(sd.SalesAmount)) AS SalesCommission
FROM 
    SalesDetails sd
JOIN HumanResources.Employee e ON sd.EmployeeID = e.EmployeeID
GROUP BY 
    e.EmployeeFirstName, 
    e.EmployeeLastName,
    e.EmployeeRegion
ORDER BY 
    TotalSalesAmount DESC;


Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

(9 rows affected)

Total execution time: 00:00:00.064

EmployeeName,NumberOfProductsSold,TotalSalesAmount,SalesCommission
Yael Peled,53,53083.10315000000,2654.16
Sara Davis,40,38758.75515000000,1937.94
Maria Cameron,34,23152.18720000000,1157.61
Don Funk,31,22823.92360000000,1141.20
Sven Mortensen,19,21929.38720000000,1096.47
Judy Lew,37,19221.72160000000,961.09
Russell King,23,18076.07360000000,903.80
Paul Suurs,27,17720.21505000000,886.01
Patricia Doyle,12,11350.98815000000,567.55


\*\*Complex Prompt 6 :  
\*\*The NorthWinds company has been operating for several years and is interested in understanding the trends of products they are selling. They want to identify the top 3 categories that have generated the most revenue in the year 2015. For each of these categories, they also want to know the top-selling product (in terms of revenue) and its corresponding supplier. 

**Database :** <span style="color: rgb(33, 33, 33); font-family: Consolas, &quot;Courier New&quot;, monospace; font-size: 12px; white-space: pre;">NorthWinds2022TSQLV7</span>

**Tables Involved** :

Sales.OrderDetail, Sales.\[Order\], Production.Product,

Production.Category, Production.Supplier, ProductRevenue

**CTE :** <span style="color: rgb(33, 33, 33); font-family: Consolas, &quot;Courier New&quot;, monospace; font-size: 12px; white-space: pre;">ProductRevenue</span>

**FUNCTION :**  <span style="color: rgb(33, 33, 33); font-family: Consolas, &quot;Courier New&quot;, monospace; font-size: 12px; white-space: pre;">dbo.fnCalculatePopularityIndex : </span> <span style="color: rgb(0, 128, 0); font-family: Consolas, &quot;Courier New&quot;, monospace; font-size: 12px; white-space: pre;">function to compute the PopularityIndex for a product.</span>

In [73]:
-- Set the active database to NorthWinds2022TSQLV7
USE NorthWinds2022TSQLV7;
GO

-- Drop the function if it exists, to avoid potential re-creation issues.
IF OBJECT_ID('dbo.fnCalculatePopularityIndex', 'FN') IS NOT NULL
    DROP FUNCTION dbo.fnCalculatePopularityIndex;
GO

-- Create a user-defined function to compute the PopularityIndex for a product.
CREATE FUNCTION dbo.fnCalculatePopularityIndex (@TotalSalesRevenue DECIMAL(18,2), @UnitPrice DECIMAL(18,2))
RETURNS DECIMAL(18,2)
AS
BEGIN
    DECLARE @Result DECIMAL(18,2)

    IF @UnitPrice = 0
        SET @Result = 0
    ELSE
        SET @Result = @TotalSalesRevenue / @UnitPrice
        
    RETURN @Result
END;
GO

-- Using a Common Table Expression (CTE) to compute total sales revenue for each product in each category.
WITH ProductRevenue AS
(
    SELECT 
        pc.CategoryName,
        p.ProductName,
        p.ProductID,
        p.UnitPrice,
        s.SupplierCompanyName,
        SUM(od.UnitPrice * od.Quantity * (1 - od.DiscountPercentage / 100)) AS SalesRevenue
    FROM Sales.OrderDetail od
    JOIN Sales.[Order] o ON od.OrderID = o.OrderID
    JOIN Production.Product p ON od.ProductID = p.ProductID
    JOIN Production.Category pc ON p.CategoryID = pc.CategoryID
    JOIN Production.Supplier s ON p.SupplierID = s.SupplierID
    WHERE YEAR(o.OrderDate) = 2015
    GROUP BY pc.CategoryName, p.ProductName, p.ProductID, p.UnitPrice, s.SupplierCompanyName
)

-- Main query to fetch the required information.
SELECT 
    pr.CategoryName,
    pr.ProductName,
    pr.SupplierCompanyName,
    pr.SalesRevenue,
    -- Use the user-defined function to calculate PopularityIndex.
    dbo.fnCalculatePopularityIndex(pr.SalesRevenue, pr.UnitPrice) AS PopularityIndex
FROM 
(
    SELECT 
        CategoryName, 
        MAX(SalesRevenue) AS MaxRevenue
    FROM ProductRevenue
    GROUP BY CategoryName
    ORDER BY MAX(SalesRevenue) DESC
    OFFSET 0 ROWS
    FETCH NEXT 3 ROWS ONLY
) AS TopCategories
JOIN ProductRevenue pr ON TopCategories.CategoryName = pr.CategoryName AND TopCategories.MaxRevenue = pr.SalesRevenue
ORDER BY 
    pr.SalesRevenue DESC, 
    pr.CategoryName;


Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

(3 rows affected)

Total execution time: 00:00:00.089

CategoryName,ProductName,SupplierCompanyName,SalesRevenue,PopularityIndex
Beverages,Product QDOMO,Supplier LVJUA,51934.55885000000,197.10
Dairy Products,Product UKXRI,Supplier OAVQT,37895.58300000000,689.01
Meat/Poultry,Product VJXYN,Supplier SVIYA,36179.79733000000,292.27


**Complex Prompt 7 :**   
As the NorthWinds company expands its operations, there is an increasing need to evaluate supplier performance. Using data from 2014, determine which suppliers are most critical based on the demand for their products. Evaluate the top products each supplier provides and calculate a Supply Index, which is a function of the number of products a supplier has and the average sales amount of these products. A higher Supply Index indicates a more critical supplier.

**Database :** <span style="color: rgb(33, 33, 33); font-family: Consolas, &quot;Courier New&quot;, monospace; font-size: 12px; white-space: pre;">NorthWinds2022TSQLV7</span>

**Tables Involved** : 

Sales.OrderDetail, Sales.\[Order\], Production.Product, 

Production.Supplier, ProductSales 

**CTE :** <span style="color: rgb(33, 33, 33); font-family: Consolas, &quot;Courier New&quot;, monospace; font-size: 12px; white-space: pre;">ProductSales</span>

**FUNCTION :**  <span style="color: rgb(33, 33, 33); font-family: Consolas, &quot;Courier New&quot;, monospace; font-size: 12px; white-space: pre;">fnCalculateSupplyIndex</span>  **\-**     <span style="color: rgb(0, 128, 0); font-family: Consolas, &quot;Courier New&quot;, monospace; font-size: 12px; white-space: pre;">&nbsp;Supply Index for a supplier</span>

In [82]:
-- Set the active database to NorthWinds2022TSQLV7
USE NorthWinds2022TSQLV7;
GO

-- Drop the function if it exists to avoid potential re-creation issues.
IF OBJECT_ID('dbo.fnCalculateSupplyIndex', 'FN') IS NOT NULL
    DROP FUNCTION dbo.fnCalculateSupplyIndex;
GO

-- Create a user-defined function to compute the Supply Index for a supplier.
CREATE FUNCTION dbo.fnCalculateSupplyIndex (@NumProducts INT, @AvgSalesAmount DECIMAL(18,2))
RETURNS DECIMAL(18,2)
AS
BEGIN
    -- The Supply Index is the product of the number of products and their average sales amount.
    RETURN @NumProducts * @AvgSalesAmount
END;
GO

-- Using a Common Table Expression (CTE) to compute total sales for each product.
WITH ProductSales AS
(
    SELECT 
        p.ProductID,
        p.SupplierID,
        SUM(od.UnitPrice * od.Quantity * (1 - od.DiscountPercentage / 100)) AS TotalSales
    FROM Sales.OrderDetail od
    JOIN Sales.[Order] o ON od.OrderID = o.OrderID
    JOIN Production.Product p ON od.ProductID = p.ProductID
    WHERE YEAR(o.OrderDate) = 2014
    GROUP BY p.ProductID, p.SupplierID
)

-- Main query to fetch the required information.
SELECT 
    s.SupplierCompanyName,
    COUNT(ps.ProductID) AS NumberOfProducts,
    AVG(ps.TotalSales) AS AvgProductSales,
    -- Use the user-defined function to calculate Supply Index.
    dbo.fnCalculateSupplyIndex(COUNT(ps.ProductID), AVG(ps.TotalSales)) AS SupplyIndex
FROM 
    Production.Supplier s
JOIN ProductSales ps ON s.SupplierID = ps.SupplierID
GROUP BY 
    s.SupplierCompanyName
ORDER BY 
    SupplyIndex DESC;


Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

(29 rows affected)

Total execution time: 00:00:00.072

SupplierCompanyName,NumberOfProducts,AvgProductSales,SupplyIndex
Supplier LVJUA,2,16646.65120000000,33293.30
Supplier SVIYA,5,4356.71460000000,21783.55
Supplier GQRCV,5,4356.64303000000,21783.20
Supplier OAVQT,2,10103.16180000000,20206.32
Supplier KEREV,3,4341.59996666666,13024.80
Supplier JNNES,3,4232.06853333333,12696.21
Supplier OGLRK,1,9847.53750000000,9847.54
Supplier BWGYE,4,2385.03087500000,9540.12
Supplier NZLIF,3,3057.12953333333,9171.39
Supplier CIYNM,3,2514.09041666666,7542.27
